<a href="https://colab.research.google.com/github/Anni1123/MachineLearning/blob/master/Kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/income_evaluation.csv', na_values=' ?')

In [21]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [22]:
import numpy as np
import time

In [23]:
df.isna().sum()

age                   0
 workclass         1836
 fnlwgt               0
 education            0
 education-num        0
 marital-status       0
 occupation        1843
 relationship         0
 race                 0
 sex                  0
 capital-gain         0
 capital-loss         0
 hours-per-week       0
 native-country     583
 income               0
dtype: int64

In [24]:
df.fillna('missing',inplace=True)

In [25]:
df.isna().sum()

age                0
 workclass         0
 fnlwgt            0
 education         0
 education-num     0
 marital-status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital-gain      0
 capital-loss      0
 hours-per-week    0
 native-country    0
 income            0
dtype: int64

In [26]:
X = df.drop(' income', axis=1)
y = df[' income']

In [27]:
y.shape

(32561,)

In [28]:
y.value_counts()

 <=50K    24720
 >50K      7841
Name:  income, dtype: int64

#KFold

In [29]:
from sklearn.model_selection import KFold

In [30]:
kf=KFold(n_splits=5, shuffle=False, random_state=None)

In [31]:
i=0
for train_set ,test_set in  kf.split(X=X):
   print(train_set,"having",len(train_set))
   print(test_set, " having :",len(test_set))
   i+=1;
print(i)

[ 6513  6514  6515 ... 32558 32559 32560] having 26048
[   0    1    2 ... 6510 6511 6512]  having : 6513
[    0     1     2 ... 32558 32559 32560] having 26049
[ 6513  6514  6515 ... 13022 13023 13024]  having : 6512
[    0     1     2 ... 32558 32559 32560] having 26049
[13025 13026 13027 ... 19534 19535 19536]  having : 6512
[    0     1     2 ... 32558 32559 32560] having 26049
[19537 19538 19539 ... 26046 26047 26048]  having : 6512
[    0     1     2 ... 26046 26047 26048] having 26049
[26049 26050 26051 ... 32558 32559 32560]  having : 6512
5


In [32]:
num_column=X.select_dtypes(include=np.number).columns
num_column

Index(['age', ' fnlwgt', ' education-num', ' capital-gain', ' capital-loss',
       ' hours-per-week'],
      dtype='object')

In [33]:
from sklearn.preprocessing import OneHotEncoder,RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [34]:
cat_column=X.select_dtypes(exclude=np.number).columns
cat_column

Index([' workclass', ' education', ' marital-status', ' occupation',
       ' relationship', ' race', ' sex', ' native-country'],
      dtype='object')

In [35]:
ct = ColumnTransformer([
    ('rob', RobustScaler(), num_column),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'), cat_column)
])


In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
pipe = Pipeline([
    ('ct_step', ct),
    ('model', RandomForestClassifier(n_estimators=10, random_state=0))
])

In [38]:
scores = []
i = 1
for train_set, test_set in kf.split(X):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1


iteration  1
iteration  2
iteration  3
iteration  4
iteration  5


In [39]:
np.array(scores)

array([0.84784278, 0.84520885, 0.84613022, 0.84858722, 0.85165848])

In [40]:
np.array(scores).mean()


0.8478855085142512

In [41]:
np.array(scores).std()

0.0022349531977626388

In [42]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold()

In [43]:
scores_skf = []
i = 1
for train_set, test_set in skf.split(X,y):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    scores_skf.append(sco)
    print("iteration ", i)
    i += 1


iteration  1
iteration  2
iteration  3
iteration  4
iteration  5


In [44]:
np.array(scores_skf)

array([0.84738216, 0.84321253, 0.84213759, 0.84259828, 0.85365479])

In [45]:
np.array(scores_skf).mean()

0.8457970705725197

In [46]:
np.array(scores_skf).std()

0.004348796764874436

In [71]:
scores_skf = []
i = 1
for train_set, test_set in skf.split(X=X,y=y):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    print("y train counts",y[train_set].value_counts())
    print("y test counts",y[test_set].value_counts())
    scores_skf.append(sco)
    print("iteration ", i)
    i += 1

y train counts  <=50K    19776
 >50K      6272
Name:  income, dtype: int64
y test counts  <=50K    4944
 >50K     1569
Name:  income, dtype: int64
iteration  1
y train counts  <=50K    19776
 >50K      6273
Name:  income, dtype: int64
y test counts  <=50K    4944
 >50K     1568
Name:  income, dtype: int64
iteration  2
y train counts  <=50K    19776
 >50K      6273
Name:  income, dtype: int64
y test counts  <=50K    4944
 >50K     1568
Name:  income, dtype: int64
iteration  3
y train counts  <=50K    19776
 >50K      6273
Name:  income, dtype: int64
y test counts  <=50K    4944
 >50K     1568
Name:  income, dtype: int64
iteration  4
y train counts  <=50K    19776
 >50K      6273
Name:  income, dtype: int64
y test counts  <=50K    4944
 >50K     1568
Name:  income, dtype: int64
iteration  5


In [63]:
from sklearn.model_selection import  cross_val_score, LeaveOneOut,RepeatedKFold,train_test_split

In [64]:
result_kf = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=5)


In [65]:
result_kf

array([0.84738216, 0.84321253, 0.84213759, 0.84259828, 0.85365479])

In [51]:
start = time.time()
result_kf10 = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=KFold(n_splits=10))
result_kf10
print("time taken: ", time.time()-start)

time taken:  6.760603904724121


In [52]:
result_kf10


array([0.83880872, 0.85165848, 0.84981572, 0.84367322, 0.85135135,
       0.84613022, 0.84520885, 0.84797297, 0.8544226 , 0.84459459])

In [53]:
start = time.time()
result_loocv = cross_val_score(estimator=pipe, X=X.head(100), y=y.head(100),
                               scoring='accuracy', cv=LeaveOneOut())
print("time taken: ", time.time()-start)

time taken:  3.1375362873077393


In [73]:
result_loocv.mean()
result_loocv

array([1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.])

In [55]:
start = time.time()
result_rkf = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy',
                              cv=RepeatedKFold(n_splits=5, n_repeats=5))
result_rkf
print("time taken: ", time.time()-start)

time taken:  14.464894771575928


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [68]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [69]:
start = time.time()
result_tts = cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='accuracy', cv=KFold(n_splits=5))
print("time taken: ", time.time()-start)

time taken:  2.355781078338623


In [58]:
result_tts


array([0.85547025, 0.84702495, 0.84184261, 0.84661163, 0.84968324])

In [59]:
pipe.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('ct_step',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('rob',
                                                  RobustScaler(copy=True,
                                                               quantile_range=(25.0,
                                                                               75.0),
                                                               with_centering=True,
                                                               with_scaling=True),
                                                  Index(['age', ' fnlwgt', ' education-num', ' capital-gain', ' capital-loss',
       ' hours-per-week'],
      dtype='object')),
                                                 ('ohe...
                 RandomForestClassifier(bootstrap=True, ccp_a

In [60]:
pipe.score(X_test, y_test)


0.8446184553968985

In [66]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']